In [0]:
dbutils.library.restartPython()

In [0]:
from pyspark.sql import functions as F
import pandas as pd
import numpy as np
import json
import re
from collections import Counter
from transformers import pipeline
import gc

2026-01-31 12:20:28.384887: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2026-01-31 12:20:28.411439: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2026-01-31 12:20:28.489263: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2026-01-31 12:20:30.522380: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


## Select Property ID

In [0]:
PROPERTY_ID = "6943815" 

## Get Price Predictions

In [0]:

# config
storage_account = "lab94290"  
container = "submissions"
sas_token ="sp=rle&st=2026-01-25T10:55:58Z&se=2026-03-01T19:10:58Z&spr=https&sv=2024-11-04&sr=c&sig=jgt2r2TSHpDaCyEfTEgHAfkvEvy49xReFDS4Mg9KnOA%3D"
sas_token = sas_token.lstrip('?')
spark.conf.set(f"fs.azure.account.auth.type.{storage_account}.dfs.core.windows.net", "SAS")
spark.conf.set(f"fs.azure.sas.token.provider.type.{storage_account}.dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.sas.FixedSASTokenProvider")
spark.conf.set(f"fs.azure.sas.fixed.token.{storage_account}.dfs.core.windows.net", sas_token)

path = f"abfss://{container}@{storage_account}.dfs.core.windows.net/Adi_Maya_Paz/predictions.csv"
predictions = spark.read.csv(path, header=True, inferSchema=True)

In [0]:
property_df = predictions.filter(F.col("property_id") == PROPERTY_ID)

In [0]:
display(property_df)

property_id,prediction
6943815,871.4430963622833


##  Review Intelligence

In [0]:
sas_token_airbnb = "sp=rle&st=2025-12-24T17:37:04Z&se=2026-02-28T01:52:04Z&spr=https&sv=2024-11-04&sr=c&sig=a0lx%2BS6PuS%2FvJ9Tbt4NKdCJHLE9d1Y1D6vpE1WKFQtk%3D"
spark.conf.set(f"fs.azure.sas.fixed.token.{storage_account}.dfs.core.windows.net", sas_token_airbnb)

# Read and Filter
df_airbnb = spark.read.parquet(f"abfss://airbnb@{storage_account}.dfs.core.windows.net/airbnb_1_12_parquet")
df_property = df_airbnb.filter(df_airbnb['property_id'] == PROPERTY_ID).toPandas()

if len(df_property) == 0:
    print("❌ No data found for this property.")
else:
    # --- FIX: COMBINE REVIEWS FROM MULTIPLE ROWS ---
    all_reviews_text = []
    
    print(f"   Found {len(df_property)} rows of data for this property.")
    
    for index, row in df_property.iterrows():
        reviews_str = row['reviews']
        if pd.notna(reviews_str):
            try:
                # Parse JSON
                batch = json.loads(reviews_str)
                if isinstance(batch, list):
                    # Clean and add to master list
                    cleaned_batch = [r.replace('<br/>', ' ').strip() for r in batch if r and len(r) > 10]
                    all_reviews_text.extend(cleaned_batch)
            except:
                continue

    print(f"✅ Successfully combined {len(all_reviews_text)} total reviews.")

    # Create the DataFrame for the next steps
    reviews_exploded = pd.DataFrame({'review_text': all_reviews_text})
    
    # Check the result
    print(f"✅ Created DataFrame with {len(reviews_exploded)} rows")

   Found 5 rows of data for this property.
✅ Successfully combined 119 total reviews.
✅ Created DataFrame with 119 rows


###  Sentiment Analysis

In [0]:
df=df_property
if len(df) == 0:
    print("❌ No review data found for this Property ID.")
else:
    
    def parse_reviews_text(reviews_str):
        if pd.isna(reviews_str): return []
        try:
            reviews = json.loads(reviews_str)
            return [r.replace('<br/>', ' ').strip() for r in reviews if r] if isinstance(reviews, list) else []
        except: return []

    def parse_category_ratings(rating_str):
        if pd.isna(rating_str): return {}
        try:
            ratings = json.loads(rating_str) if isinstance(rating_str, str) else rating_str
            return {item['name']: float(item['value']) for item in ratings if 'name' in item} if isinstance(ratings, list) else {}
        except: return {}

    # Process Columns
    df['reviews_list'] = df['reviews'].apply(parse_reviews_text)
    df['ratings_dict'] = df['category_rating'].apply(parse_category_ratings)
    
    # Extract specific ratings
    for cat in ['Cleanliness', 'Communication', 'Check-in', 'Location', 'Value']:
        df[f'{cat}_rating'] = df['ratings_dict'].apply(lambda x: x.get(cat, 0.0))

    # Explode reviews for analysis
    reviews_exploded = df.explode('reviews_list').rename(columns={'reviews_list': 'review_text'})
    reviews_exploded = reviews_exploded[reviews_exploded['review_text'].notna() & (reviews_exploded['review_text'].str.len() > 10)]

### SENTIMENT ANALYSIS (DistilBERT)

In [0]:
print("Running Sentiment Analysis...")
sentiment_analyzer = pipeline("sentiment-analysis", model="distilbert-base-uncased-finetuned-sst-2-english", truncation=True, max_length=256)

def get_sentiment(text):
    try:
        # Truncate text to 256 chars to fit model limit
        res = sentiment_analyzer(str(text)[:256])[0]
        return res['score'] if res['label'] == 'POSITIVE' else -res['score']
    except: 
        return 0.0

reviews_exploded['sentiment_score'] = reviews_exploded['review_text'].apply(get_sentiment)
reviews_exploded['sentiment_label'] = reviews_exploded['sentiment_score'].apply(lambda x: 'POSITIVE' if x > 0 else 'NEGATIVE')

Running Sentiment Analysis...


### THEME EXTRACTION & TIPS

In [0]:

print("🔍 Extracting themes and searching knowledge base...")

ISSUE_PATTERNS = {
    # ============ NEGATIVE ISSUES ============
    "wifi_issues": [r"\bwifi slow\b", r"\bwifi didn'?t work\b", r"\bno wifi\b", r"\binternet slow\b", r"\bwifi bad\b", r"\bweak wifi\b", r"\bpoor wifi\b", r"\bno internet\b"],
    "dirty_general": [r"\bdirty\b", r"\bnot clean\b", r"\bunclean\b", r"\bfilthy\b", r"\bdusty\b", r"\bneeds cleaning\b"],
    "dirty_bathroom": [r"\bdirty bathroom\b", r"\bbathroom not clean\b", r"\bmold\b", r"\bmildew\b", r"\btoilet dirty\b"],
    "dirty_kitchen": [r"\bdirty kitchen\b", r"\bdirty dishes\b", r"\bkitchen not clean\b", r"\bgreasy\b"],
    "dirty_bedding": [r"\bdirty sheets\b", r"\bstained sheets\b", r"\bdirty towels\b", r"\bstained towels\b"],
    "bugs_pests": [r"\bbugs\b", r"\binsects\b", r"\bants\b", r"\bcockroach", r"\broaches\b", r"\bmice\b", r"\bbed bugs\b"],
    "smell_issues": [r"\bbad smell\b", r"\bsmelled bad\b", r"\bodor\b", r"\bmusty\b", r"\bsmoke smell\b", r"\bsmelly\b"],
    "noise_general": [r"\bnoisy\b", r"\btoo loud\b", r"\bnot quiet\b"],
    "noise_street": [r"\bstreet noise\b", r"\btraffic noise\b", r"\bnoisy street\b", r"\bloud traffic\b"],
    "noise_neighbors": [r"\bloud neighbors\b", r"\bnoisy neighbors\b", r"\bthin walls\b", r"\bcould hear neighbors\b"],
    "too_hot": [r"\btoo hot\b", r"\bno ac\b", r"\bac didn'?t work\b", r"\bac broken\b", r"\bno air conditioning\b"],
    "too_cold": [r"\btoo cold\b", r"\bno heating\b", r"\bfreezing\b", r"\bheater broken\b", r"\bno heat\b"],
    "uncomfortable_bed": [r"\buncomfortable bed\b", r"\bbad mattress\b", r"\bhard mattress\b", r"\blumpy mattress\b"],
    "hot_water_issues": [r"\bno hot water\b", r"\bcold water only\b", r"\bwater pressure low\b", r"\bhot water ran out\b"],
    "plumbing_issues": [r"\btoilet broken\b", r"\bclogged drain\b", r"\bsink clogged\b", r"\bleaking\b", r"\bdripping\b"],
    "checkin_problems": [r"\bcheck-?in problem\b", r"\bcheck-?in difficult\b", r"\bcouldn'?t check in\b"],
    "hard_to_find": [r"\bhard to find\b", r"\bcouldn'?t find\b", r"\bwrong address\b", r"\bconfusing directions\b", r"\bgot lost\b"],
    "key_issues": [r"\bkey didn'?t work\b", r"\blockbox problem\b", r"\bcouldn'?t get in\b", r"\blocked out\b", r"\bwrong code\b"],
    "host_unresponsive": [r"\bhost didn'?t respond\b", r"\bno response\b", r"\bslow response\b", r"\bcouldn'?t reach host\b"],
    "overpriced": [r"\boverpriced\b", r"\btoo expensive\b", r"\bnot worth the price\b", r"\bnot worth\b"],
    "too_small": [r"\btoo small\b", r"\btiny\b", r"\bcramped\b", r"\bsmaller than expected\b"],
    "safety_concerns": [r"\bdidn'?t feel safe\b", r"\bunsafe\b", r"\bsketchy area\b", r"\bdangerous\b"],
    
    # ============ POSITIVE THEMES ============
    "great_location": [r"\bgreat location\b", r"\bperfect location\b", r"\bamazing location\b", r"\bbest location\b", r"\bexcellent location\b"],
    "walkable": [r"\bwalking distance\b", r"\bwalkable\b", r"\bclose to everything\b", r"\bsteps away\b", r"\beasy walk\b"],
    "quiet_area": [r"\bquiet area\b", r"\bpeaceful\b", r"\bquiet neighborhood\b", r"\bvery quiet\b", r"\btranquil\b"],
    "central": [r"\bcentral location\b", r"\bin the center\b", r"\bheart of the city\b", r"\bdowntown\b", r"\bcity center\b"],
    "very_clean": [r"\bvery clean\b", r"\bspotless\b", r"\bimmaculate\b", r"\bsuper clean\b", r"\bsparkling clean\b", r"\bpristine\b"],
    "great_host": [r"\bgreat host\b", r"\bamazing host\b", r"\bwonderful host\b", r"\bbest host\b", r"\bfantastic host\b", r"\blovely host\b"],
    "responsive_host": [r"\bresponsive\b", r"\bquick response\b", r"\bfast response\b", r"\bresponded quickly\b", r"\bimmediate response\b"],
    "friendly_host": [r"\bfriendly host\b", r"\bwelcoming host\b", r"\bwarm welcome\b", r"\bhost was friendly\b", r"\bhost was kind\b"],
    "great_communication": [r"\bgreat communication\b", r"\beasy communication\b", r"\bclear communication\b", r"\bgood communication\b"],
    "great_view": [r"\bgreat view\b", r"\bamazing view\b", r"\bbeautiful view\b", r"\bstunning view\b", r"\bincredible view\b"],
    "beautiful_place": [r"\bbeautiful place\b", r"\bgorgeous\b", r"\bstunning\b", r"\blovely decor\b", r"\bbeautifully decorated\b", r"\bcharming\b"],
    "good_value": [r"\bgreat value\b", r"\bgood value\b", r"\bworth the price\b", r"\bgood price\b", r"\bexcellent value\b", r"\baffordable\b"],
    "well_equipped": [r"\bwell equipped\b", r"\bhad everything\b", r"\bfully equipped\b", r"\ball amenities\b", r"\beverything we needed\b"],
    "great_kitchen": [r"\bgreat kitchen\b", r"\bfull kitchen\b", r"\bwell equipped kitchen\b", r"\bkitchen had everything\b"],
    "good_bed": [r"\bcomfortable bed\b", r"\bgreat bed\b", r"\bslept well\b", r"\bcomfy bed\b", r"\bgood mattress\b"],
    "good_wifi": [r"\bfast wifi\b", r"\bgreat wifi\b", r"\bwifi worked\b", r"\bstrong wifi\b", r"\bgood internet\b"],
    "very_comfortable": [r"\bvery comfortable\b", r"\bsuper comfortable\b", r"\bso comfortable\b", r"\bcozy\b", r"\bhomey\b", r"\bfelt like home\b"],
    "spacious": [r"\bspacious\b", r"\blots of space\b", r"\bplenty of room\b", r"\broomy\b", r"\bbig apartment\b"],
    "will_return": [r"\bwill be back\b", r"\bcome back\b", r"\breturn\b", r"\bstay again\b", r"\bbook again\b", r"\bdefinitely return\b"],
    "highly_recommend": [r"\bhighly recommend\b", r"\bdefinitely recommend\b", r"\bwould recommend\b", r"\bstrongly recommend\b"],
    "exceeded_expectations": [r"\bexceeded expectations\b", r"\bbetter than expected\b", r"\babove and beyond\b", r"\bblown away\b"],
    "perfect_stay": [r"\bperfect stay\b", r"\bamazing stay\b", r"\bwonderful stay\b", r"\bfantastic stay\b", r"\bbest stay\b"],
    "easy_checkin": [r"\beasy check-?in\b", r"\bsmooth check-?in\b", r"\bsimple check-?in\b", r"\bself check-?in\b", r"\bseamless\b"],
}

NEGATIVE_THEMES = [
    "wifi_issues", "dirty_general", "dirty_bathroom", "dirty_kitchen", "dirty_bedding",
    "bugs_pests", "smell_issues", "noise_general", "noise_street", "noise_neighbors",
    "too_hot", "too_cold", "uncomfortable_bed", "hot_water_issues", "plumbing_issues",
    "checkin_problems", "hard_to_find", "key_issues", "host_unresponsive", "overpriced",
    "too_small", "safety_concerns"
]


ISSUE_KEYWORDS = {
    "wifi_issues": ["wifi", "internet", "connection", "network", "speed"],
    "dirty_general": ["clean", "declutter", "tidy", "spotless", "cleaning"],
    "dirty_bathroom": ["bathroom", "toilet", "shower", "towel", "mold"],
    "dirty_kitchen": ["kitchen", "dishes", "cooking", "grease", "utensils"],
    "dirty_bedding": ["sheet", "linen", "bedding", "pillow", "towel"],
    "bugs_pests": ["pest", "bug", "insect", "ant", "roach"],
    "smell_issues": ["smell", "odor", "fresh", "ventilation", "scent"],
    "noise_general": ["noise", "quiet", "sound", "peaceful", "party"],
    "noise_street": ["street", "traffic", "window", "insulation"],
    "noise_neighbors": ["neighbor", "wall", "sound", "community"],
    "too_hot": ["ac", "air conditioning", "cool", "temperature", "fan"],
    "too_cold": ["heat", "heating", "warm", "temperature", "winter"],
    "uncomfortable_bed": ["bed", "mattress", "sleep", "pillow", "comfort"],
    "hot_water_issues": ["water", "heater", "shower", "plumbing"],
    "plumbing_issues": ["leak", "drain", "clog", "toilet", "sink"],
    "checkin_problems": ["check-in", "checkin", "arrival", "access", "lock"],
    "hard_to_find": ["direction", "find", "map", "address", "location"],
    "key_issues": ["key", "lockbox", "code", "entry", "access"],
    "host_unresponsive": ["response", "respond", "communication", "message"],
    "overpriced": ["price", "value", "cost", "expensive"],
    "too_small": ["space", "size", "cramped", "photo", "accurate"],
    "safety_concerns": ["safe", "safety", "lock", "security", "danger"]
}

def extract_themes(text):
    text_lower = str(text).lower()
    found_themes = []
    for theme, patterns in ISSUE_PATTERNS.items():
        for p in patterns:
            if re.search(p, text_lower):
                found_themes.append(theme)
                break
    return found_themes

reviews_exploded['themes'] = reviews_exploded['review_text'].apply(extract_themes)

all_tips = []
try:
        with open('scraping_results_full_english.json', 'r') as f:
            knowledge_base_raw = json.load(f)
            
        for article in knowledge_base_raw:
            if article.get("status") != "success": continue
            
            content = article.get("content", {})
            title = content.get("page_title", "Airbnb Help")
            
            # 1. Extract from list items (Original Logic)
            for item in content.get("list_items", []):
                if 30 < len(item) < 600:
                    skip_words = ['privacy', 'footer', 'help center', 'careers', 'newsroom', 'investors']
                    if not any(skip in item.lower() for skip in skip_words):
                        all_tips.append({'tip': item, 'source': title})
            
            # 2. Extract from paragraphs (Original Logic)
            for para in content.get("paragraphs", []):
                if 50 < len(para) < 500:
                    skip_words = ['privacy', 'footer', 'help center']
                    if not any(skip in para.lower() for skip in skip_words):
                        all_tips.append({'tip': para, 'source': title})
                        
        print(f"✅ Loaded {len(all_tips)} potential tips from knowledge base.")
        
except Exception as e:
        print(f"⚠️ Could not load scraping_results_full_english.json: {e}")
        all_tips = []

def get_dynamic_suggestion(issue_name):
    # Search the loaded tips for the best match
    if issue_name not in ISSUE_KEYWORDS: return None
    
    keywords = ISSUE_KEYWORDS[issue_name]
    best_match = None
    highest_score = 0
    
    for tip_entry in all_tips:
        tip_text = tip_entry['tip'].lower()
        # Score = how many keywords appear in the tip
        score = sum(1 for kw in keywords if kw in tip_text)
        
        if score > highest_score:
            highest_score = score
            best_match = tip_entry
    
    return best_match

🔍 Extracting themes and searching knowledge base...
✅ Loaded 1251 potential tips from knowledge base.


### AGGREGATE REPORT

In [0]:
pos_reviews = reviews_exploded[reviews_exploded['sentiment_label'] == 'POSITIVE']
neg_reviews = reviews_exploded[reviews_exploded['sentiment_label'] == 'NEGATIVE']

# Count Themes
all_themes = [t for sublist in reviews_exploded['themes'] for t in sublist]
theme_counts = Counter(all_themes)

advantages = [t for t, c in theme_counts.most_common() if t not in NEGATIVE_THEMES][:5]
disadvantages = [t for t, c in theme_counts.most_common() if t in NEGATIVE_THEMES][:5]

# Generate Suggestions using Dynamic Search
suggestions = []
sources = set()

for issue in disadvantages:
    match = get_dynamic_suggestion(issue)
    if match:
        suggestions.append(f"[{issue.replace('_', ' ').title()}]: {match['tip']}")
        sources.add(match['source'])
    else:
        suggestions.append(f"[{issue.replace('_', ' ').title()}]: Check listing details.")

suggestion_str = " || ".join(suggestions) if suggestions else "No specific issues identified - keep up the great work!"
source_str = " | ".join(sources) if sources else "Airbnb Knowledge Base"


### PRINT REPORT

In [0]:


print("\n" + "="*50)
print("📋 GUEST FEEDBACK INTELLIGENCE REPORT")
print("="*50)

# Safe retrieval of host info
seller_info_str = df['seller_info'].iloc[0] if 'seller_info' in df.columns and pd.notna(df['seller_info'].iloc[0]) else "{}"
try:
    host_id = json.loads(seller_info_str).get('seller_id', 'Unknown')
except:
    host_id = 'Unknown'

print(f" Property ID: {PROPERTY_ID}")
print(f" Host ID: {host_id}")
print(f" Listing Name: {df['listing_name'].iloc[0]}")
print(f" Location: {df['location'].iloc[0]}")
print(f" Total Reviews: {len(reviews_exploded)}")
print(f" Positive Reviews: {len(pos_reviews)}")
print(f" Negative Reviews: {len(neg_reviews)}")
print(f" Avg Sentiment Score: {reviews_exploded['sentiment_score'].mean():.3f}")

for col in ['Cleanliness', 'Communication', 'Check-in', 'Location', 'Value']:
    col_name = f"{col}_rating"
    if col_name in df.columns:
        print(f" {col} Rating: {df[col_name].iloc[0]}")

print(f" Advantages: {', '.join([f'{t} ({theme_counts[t]}x)' for t in advantages])}")
print(f" Disadvantages: {', '.join([f'{t} ({theme_counts[t]}x)' for t in disadvantages])}")
print(f" Suggestions: {suggestion_str}")
print(f" Suggestion Sources: {source_str}")


📋 GUEST FEEDBACK INTELLIGENCE REPORT
 Property ID: 6943815
 Host ID: 21271697
 Listing Name: Entire rental unit in Paris, France
 Location: Paris, Île-de-France, France
 Total Reviews: 119
 Positive Reviews: 112
 Negative Reviews: 7
 Avg Sentiment Score: 0.884
 Cleanliness Rating: 4.8
 Communication Rating: 4.9
 Check-in Rating: 4.9
 Location Rating: 4.9
 Value Rating: 4.7
 Advantages: great_location (36x), responsive_host (28x), highly_recommend (17x), very_comfortable (13x), walkable (13x)
 Disadvantages: smell_issues (3x), dirty_general (2x), noise_general (2x)
 Suggestions: [Smell Issues]: Everyone has a different sense of smell. Strong scents of any kind can be off-putting to guests. Trying to mask odors with bleach, air fresheners, or other solutions usually doesn’t work, Diana says. || [Dirty General]: Clean and declutter. Show guests exactly what they can expect. “It’s surprising how many people forget to clear their spaces of random objects before taking photos,” says Jeff, a